Ok, so let's test out some stuff, and use this as a notebook for what works and what doesn't.

### Idea 1: Momentum of Some Sort (MACD, RSI, etc.)

Don't have a lot of confidence in this tbh since crypto seems to be trading with more volatility. Let's test, though!

Let's first try a MACD-- comparing the (difference of long and short EMAs) and the EMA of this difference. We compare the difference with its own EMA because if not, there would be too many crossovers (> 0 vs < 0), and too many damn orders going through. Comparing against the EMA makes much fewer crossovers, but also means your signal is now acceleration instead of velocity [site intuition]

We also note that if we only take the sign of the difference as our signal, we will be too late-- our signal will be incredibly lagged and we won't have the adequate time to capitalize on trends before they swing again. Comparing with the EMA of difference will make a huge difference. See below. Tho to note, the histogram is just the difference between the two...

![Chart](MACDpicwiki.gif "hi")

Let's implement this, then, with long window = 13 and short window = 6, EMA of Diff = 6
data in 15 min. increments

We use $ \alpha = \frac{2}{N+1} $, and seek 99% accuracy

the general idea is to long/short when diff of MACD and MACD Signal (the histogram) crosses 0, depending on the direction of course, and exit after histogram peaks. So, liquidate the 15 minute interval after histogram relative extrema. Repeat when histogram crosses 0 again.

Furthermore, after reviewing the first batch of backtesting, I've noticed that this program often exits in the early stages of big swings, and so only captures a tiny fraction of the possible gain. I am working to make the strategy re-enter a position given a histogram swing in the top xth percentile.

Inhomogeneous time series if we want to look at imperfect data (with holes and all), but we're not doing that

Now, considering a much much more shorter window (5 min positions), and am adjusting the windows accordingly (3, 10, 3)
I think we may have found our strategy...

Fuck it just do testing on main and descriptions here smh
From 6/30/2021 - 6/30/2022 (accounting Kraken fees)
[Excess Mean, Vol, Sharpe] = [1.005296,  1.363965,  0.73704]
From 6/30/2020 - 6/30/2021
[Excess Mean, Vol, Sharpe] = [-0.349, 0.251, -1.389]
a tale of two returns...
over past 8 years, avg. of -30%... uh
^ Outdated Figures

anyways, there are some bigggg problems with this strategy-- significantly, that it's on kraken and fees are eating my ass

# Idea 2: transfer to Binance.US lol #
